In [1]:
!pip install transformers==4.30
!pip install accelerate -U
!pip install rouge_score
!pip install datasets
!pip install evaluate
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.3
    Uninstalling transformers-4.39.3:
      Successfully uninstalled transformers-4.39.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.14.3 requires transformers>=4.33.1, but you have transformers 4.30.0 which is incompatible.
  Preparing metadata (setup.py) ... - done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size

In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, MT5ForConditionalGeneration, MT5Tokenizer
from datasets import Dataset
import numpy as np
import nltk
import evaluate
rouge_score = evaluate.load("rouge")
nltk.download("punkt")
from transformers import DataCollatorForSeq2Seq
from nltk.tokenize import sent_tokenize

2024-04-24 22:21:46.410331: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 22:21:46.410455: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 22:21:46.541458: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
!pip install indic-nlp-library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 5.8 MB/s eta 0:00:00


In [4]:
!pip install datasets transformers accelerate evaluate sentencepiece

In [5]:
!pip install sentencepiece

In [6]:
!pip install sentencepiece

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
from transformers import MBartForConditionalGeneration, AutoModelForSeq2SeqLM
from transformers import AlbertTokenizer, AutoTokenizer
model = "google/mt5-base"
tokenizer = AutoTokenizer.from_pretrained(model, 
                                               do_lower_case=False, 
                                               use_fast=False, 
                                               keep_accents=True)

model = AutoModelForSeq2SeqLM.from_pretrained(model).to(device)


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:
bos_id = tokenizer._convert_token_to_id_with_added_voc("<s>")
eos_id = tokenizer._convert_token_to_id_with_added_voc("</s>")
pad_id = tokenizer._convert_token_to_id_with_added_voc("<pad>")


In [10]:
train_path="/kaggle/input/text-rank/hindi_train_final_TEXTRANK.csv"
val_path="/kaggle/input/text-rank/hindi_val_final_TEXTRANK.csv"
test_path="/kaggle/input/text-rank/hindi_test_final_TEXTRANK.csv"
df_train=pd.read_csv(train_path)[["Article", "Summary"]]
df_val=pd.read_csv(val_path)[["Article", "Summary"]]
df_test=pd.read_csv(test_path)[["Article", "Summary"]]


In [11]:
df_train = df_train.dropna()
df_val = df_val.dropna()
df_test = df_test.dropna()

In [12]:
train_data = Dataset.from_pandas(df_train)
val_data = Dataset.from_pandas(df_val)
test_data = Dataset.from_pandas(df_test)

In [13]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [14]:
def tokenize_helper(dat):
    print("Article:", type(dat["Article"]))
    print("Summary:", type(dat["Summary"]))
    
    model_inputs = tokenizer(
        dat["Article"],
        truncation=True,
        max_length=512  # Maximum length for articles
    )
    true_sent = tokenizer(
        dat["Summary"],
        truncation=True,
        max_length=64  # Maximum length for summaries
    )
    model_inputs["labels"] = true_sent["input_ids"]
    return model_inputs



In [15]:
def add_words(example):
    # Modify the column 'column_name' by adding words to each entry
    example['input'] = [entry + "  </s> <2hi>" for entry in example['Article']]
    example['target'] = ["<2hi> " + entry + " </s>" for entry in example['Summary']]
    return example


In [16]:
train_data=train_data.map(add_words, batched=True)
val_data=val_data.map(add_words, batched=True)

Map:   0%|          | 0/19087 [00:00<?, ? examples/s]

Map:   0%|          | 0/1061 [00:00<?, ? examples/s]

In [17]:
tokenized_train = train_data.map(tokenize_helper, batched=True)
tokenized_val = val_data.map(tokenize_helper, batched=True)

Map:   0%|          | 0/19087 [00:00<?, ? examples/s]

Article: <class 'list'>
Summary: <class 'list'>
Article: <class 'list'>
Summary: <class 'list'>
Article: <class 'list'>
Summary: <class 'list'>
Article: <class 'list'>
Summary: <class 'list'>
Article: <class 'list'>
Summary: <class 'list'>
Article: <class 'list'>
Summary: <class 'list'>
Article: <class 'list'>
Summary: <class 'list'>
Article: <class 'list'>
Summary: <class 'list'>
Article: <class 'list'>
Summary: <class 'list'>
Article: <class 'list'>
Summary: <class 'list'>
Article: <class 'list'>
Summary: <class 'list'>
Article: <class 'list'>
Summary: <class 'list'>
Article: <class 'list'>
Summary: <class 'list'>
Article: <class 'list'>
Summary: <class 'list'>
Article: <class 'list'>
Summary: <class 'list'>
Article: <class 'list'>
Summary: <class 'list'>
Article: <class 'list'>
Summary: <class 'list'>
Article: <class 'list'>
Summary: <class 'list'>
Article: <class 'list'>
Summary: <class 'list'>
Article: <class 'list'>
Summary: <class 'list'>


Map:   0%|          | 0/1061 [00:00<?, ? examples/s]

Article: <class 'list'>
Summary: <class 'list'>
Article: <class 'list'>
Summary: <class 'list'>


In [18]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)
tokenized_train = tokenized_train.remove_columns(train_data.column_names)
tokenized_val = tokenized_val.remove_columns(val_data.column_names)

In [19]:
temp = tokenizer(test_data["Article"][0], truncation = True, padding=True, return_tensors='pt').input_ids.to(device)
out = model.generate(temp, max_length=150, num_beams=4, early_stopping=True) 
decoded_output = tokenizer.decode(out[0], 
                                    skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=False)

print(decoded_output)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


<extra_id_0> उल्कापिंड हो गया।


In [20]:
test_data["Article"][0]

"दरअसल, उल्कापिंड जब धरती पर गिरते हैं, तो इनकी चमक इतनी ज्यादा होती है कि लगभग  200 से 300 किलोमीटर के दायरे के लोग आसमान में इसे देख सकते हैं। वहीं लोग इस घटना को देखने के बाद कई तरह की बात कह रहे हैं। इस फुटेज को महाराष्ट्र के नागपुर और मध्य प्रदेश के झाबुआ एवं बड़वानी जिलों में देखे जाने की खबर है। माना जा रहा है बिजली की चमक जैसी ये लकीरें उल्का हैं। वैसे तो इसे उल्का पिंड बताया जा रहा है, लेकिन खगोलविदों ने अनुमान लगाया है कि उपग्रह दुर्घटनाग्रस्त हो गया होगा। हालांकि इस घटना ने सबको आश्चर्य में डाल दिया है। वहीं कुछ खगोलविदों का मानना है कि यह घटना 'चीनी रॉकेट चरण का पुन: प्रवेश' थी, जिसे फरवरी 2021 में लॉन्च किया गया था। वीडियो देखने पर ऐसा लगता है मानों कोई उल्का पिंड या पुच्छल तार हो। आम लोगों से लेकर खगोलविद तक अलग-अलग बात कह रहे हैं, परंतु इस घटना से अभी तक किसी तरह का नुकसान नहीं हुआ है।। इस रोशनी को देखने वालों की मानें तो आकाश से ज़मीन की तरफ उन्हें तेज़ रफ्तार से रोशनी आती दिखी। ये सौर मंडल में चक्कर लगाते हुए किसी भी ग्रह के वायुमंडल में प्रवेश कर सकते हैं। छोटे-छोट उल

In [21]:
batch_size = 2
epochs = 4
args = Seq2SeqTrainingArguments(
    output_dir = "/kaggle/working/",
    evaluation_strategy = "epoch",
    save_strategy='epoch',  # Save model on last epoch
    save_total_limit=1,
    learning_rate = 1e-3,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    predict_with_generate=True,
    num_train_epochs = epochs,
    report_to="none"
)

In [22]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [23]:
import torch
torch.cuda.empty_cache()

In [24]:
!rm -rf /kaggle/working/*

In [25]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,1.544600,1.436609,9.253100,2.707300,9.036400,9.093000
2,1.280900,1.265425,10.245300,3.256000,9.890500,9.943800
3,1.045000,1.210453,11.804400,3.933100,11.179800,11.247200
4,0.837700,1.237283,12.226600,4.305100,11.663100,11.753900


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=19088, training_loss=1.2769897769502703, metrics={'train_runtime': 20718.0383, 'train_samples_per_second': 3.685, 'train_steps_per_second': 0.921, 'total_flos': 9.051388478466048e+16, 'train_loss': 1.2769897769502703, 'epoch': 4.0})

In [26]:
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 1.2372831106185913,
 'eval_rouge1': 12.2266,
 'eval_rouge2': 4.3051,
 'eval_rougeL': 11.6631,
 'eval_rougeLsum': 11.7539,
 'eval_runtime': 282.3269,
 'eval_samples_per_second': 3.758,
 'eval_steps_per_second': 0.942,
 'epoch': 4.0}

In [27]:
print("model evaluation done")

model evaluation done


In [28]:
actual=[]
pred=[]
for i in range(len(test_data)):
    actual.append(test_data["Article"][i])
    temp = tokenizer(test_data["Article"][i], truncation = True, padding=True, return_tensors='pt').input_ids.to(device)
    out = model.generate(temp, max_length=150, num_beams=4, early_stopping=True) 
    decoded_output = tokenizer.decode(out[0], 
                                    skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=False)
    pred.append(decoded_output)

# print(decoded_output)

In [29]:
import pickle

In [30]:
with open("/kaggle/working/pred", 'wb') as file:
    # Dump the list into the file using pickle
    pickle.dump(pred, file)
    
    
print("predictions")

predictions


In [31]:
with open("/kaggle/working/actual", 'wb') as file:
    # Dump the list into the file using pickle
    pickle.dump(actual, file)
    
    
print("actuals")

actuals


In [32]:
print(pred)

['इस फुटेज को महाराष्ट्र के नागपुर और मध्य प्रदेश के झाबुआ एवं बड़वानी जिलों में देखे जाने की खबर है। इस फुटेज को महाराष्ट्र के नागपुर और मध्य प्रदेश के झाबुआ एवं बड़वानी जिलों में', 'रूस के डिप्टी फॉरेन मिनिस्टर ने बुधवार को ऐलान किया है कि वो अब से न्यू START ट्रीटी के तहत अपने नए परमाणु हथियारों की टेस्टिंग की जानकारी अमेरिका को नहीं देगा।', 'पूर्व रक्षा मंत्री और समाजवादी पार्टी के सरंक्षक मुलायम सिंह यादव कोरोना संक्रमित पाए गए हैं। इसकी जानकारी समाजवादी पार्टी के ट्वीटर हैंडल से ट्वीट करके दी गई है।', 'कर्नाटक में भारतीय जनता पार्टी के विधायक बसनगौड़ा पाटिल यतनाल ने दावा किया है कि कुछ लोगों ने उनसे संपर्क कर 2,500 करोड़ रुपये (2500 crore to become CM) में राज्य के मुख्यमंत्री', 'उत्तर प्रदेश में सीतापुर जिले के विकास भवन रजिस्ट्री कार्यालय में एक बंदर ने हंगामा खड़ा कर दिया। यहां बंदर एक बुजुर्ग आदमी से करेंसी नोटों से भरा बैग छीन कर पेड़ पर चढ़', 'Rajasthan Junaid Nasir Murder Case - Follow Junaid Nasir Murder Latest News, Reports and Updates On Dainik Bhaskar. सलूणी सब डिवीजन 